In [ ]:
!pip install tensorflow==1.14
# !pip install tensorflow-gpu==1.14
import tensorflow.compat.v1 as tf

In [ ]:
import json
!pip install hickle
%env SM_FRAMEWORK=tf.keras
!pip install keras==2.3.1
import os
import datetime 
import sys 
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import random
from tensorflow.keras.models import Model
import pickle
import cv2
import PIL
from PIL import Image
from scipy import ndimage
from collections import Counter
import imageio
import hickle
import os
import json
import scipy
import tensorflow as tf
import keras
import keras
from keras import backend as K
from keras import activations, initializers, regularizers, constraints, metrics
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import (Dense, Dropout, Activation, Flatten, Reshape, Layer,
                          BatchNormalization, LocallyConnected2D,
                          ZeroPadding2D, Conv2D, MaxPooling2D, Conv2DTranspose,
                          GaussianNoise, UpSampling2D, Input)
from keras.utils import conv_utils
from keras.layers import Lambda
from keras.engine import Layer, InputSpec
from tensorflow.keras.layers import MaxPooling2D,Conv2D,Input,Add,MaxPool2D,Flatten,AveragePooling2D,Dense,BatchNormalization,ZeroPadding2D,Activation,Concatenate,UpSampling2D
from tensorflow.keras.models import Model
from keras import Input

env: SM_FRAMEWORK=tf.keras


Using TensorFlow backend.


In [ ]:
!pip install "git+https://github.com/salaniz/pycocoevalcap.git"

  Cloning https://github.com/salaniz/pycocoevalcap.git to /tmp/pip-req-build-uhcuwrc9
  Running command git clone -q https://github.com/salaniz/pycocoevalcap.git /tmp/pip-req-build-uhcuwrc9
  Created wheel for pycocoevalcap: filename=pycocoevalcap-1.2-cp37-none-any.whl size=104312216 sha256=e7a2cc526fd7f6c76e875cc3da84738eb3d8e1b84fbfab0cb3ac18d5797c711e
  Stored in directory: /tmp/pip-ephem-wheel-cache-49auzye7/wheels/43/c5/1e/8c540096ae74b7789563f1ba2a0e195308ee86f445a0cda110
Successfully built pycocoevalcap


In [ ]:
# !cp -r '/content/log' '/content/drive/MyDrive/Capstone'

In [ ]:
sys.path.append('/content/drive/MyDrive/Capstone/show-attend-and-tell-master')

from collections import Counter
from core.vggnet import Vgg19
from core.utils import *
from core.solver import CaptioningSolver
from core.model import CaptionGenerator
from core.utils import load_coco_data


sys.path.append('./')

In [ ]:
!wget 'http://msvocds.blob.core.windows.net/annotations-1-0-3/captions_train-val2014.zip' -P data/
!wget 'http://www.vlfeat.org/matconvnet/models/imagenet-vgg-verydeep-19.mat' -P data/
!wget 'http://images.cocodataset.org/zips/train2014.zip' -P image/
!wget 'http://images.cocodataset.org/zips/val2014.zip' -P image/

--2021-04-15 16:15:11--  http://msvocds.blob.core.windows.net/annotations-1-0-3/captions_train-val2014.zip
Resolving msvocds.blob.core.windows.net (msvocds.blob.core.windows.net)... 52.176.224.96
Connecting to msvocds.blob.core.windows.net (msvocds.blob.core.windows.net)|52.176.224.96|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19673183 (19M) [application/octet-stream Charset=UTF-8]
Saving to: ‘data/captions_train-val2014.zip’

captions_train-val2 100%[===================>]  18.76M  1.26MB/s    in 11s     

2021-04-15 16:15:22 (1.77 MB/s) - ‘data/captions_train-val2014.zip’ saved [19673183/19673183]

--2021-04-15 16:15:22--  http://www.vlfeat.org/matconvnet/models/imagenet-vgg-verydeep-19.mat
Resolving www.vlfeat.org (www.vlfeat.org)... 64.90.48.57
Connecting to www.vlfeat.org (www.vlfeat.org)|64.90.48.57|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.vlfeat.org/matconvnet/models/imagenet-vgg-verydeep-1

In [ ]:
!unzip 'data/captions_train-val2014.zip' -d data/
!unzip 'image/train2014.zip' -d image/
!rm 'image/train2014.zip' 
!unzip 'image/val2014.zip' -d image/ 
!rm 'image/val2014.zip' 

Streaming output truncated to the last 5000 lines.
 extracting: image/val2014/COCO_val2014_000000535608.jpg  
 extracting: image/val2014/COCO_val2014_000000279145.jpg  
 extracting: image/val2014/COCO_val2014_000000243495.jpg  
 extracting: image/val2014/COCO_val2014_000000225791.jpg  
 extracting: image/val2014/COCO_val2014_000000428067.jpg  
 extracting: image/val2014/COCO_val2014_000000059383.jpg  
 extracting: image/val2014/COCO_val2014_000000558498.jpg  
 extracting: image/val2014/COCO_val2014_000000376123.jpg  
 extracting: image/val2014/COCO_val2014_000000191304.jpg  
 extracting: image/val2014/COCO_val2014_000000171062.jpg  
 extracting: image/val2014/COCO_val2014_000000014549.jpg  
 extracting: image/val2014/COCO_val2014_000000121152.jpg  
 extracting: image/val2014/COCO_val2014_000000276596.jpg  
 extracting: image/val2014/COCO_val2014_000000029431.jpg  
 extracting: image/val2014/COCO_val2014_000000036349.jpg  
 extracting: image/val2014/COCO_val2014_000000502055.jpg  
 extr

In [ ]:
def resize_image(image):
    width, height = image.size
    if width > height:
        left = (width - height) / 2
        right = width - left
        top = 0
        bottom = height
    else:
        top = (height - width) / 2
        bottom = height - top
        
        left = 0
        right = width
    image = image.crop((left, top, right, bottom))
    image = image.resize([224, 224], Image.ANTIALIAS)
    return image

In [ ]:
splits = ['train','val']

for split in splits:
    folder = './image/%s2014' %split
    resized_folder = './image/%s2014_resized/' %split
    if not os.path.exists(resized_folder):
        os.makedirs(resized_folder)
    print('Start resizing %s images.' %split)
    image_files = os.listdir(folder)
    num_images = len(image_files)
    for i, image_file in enumerate(image_files):
        with open(os.path.join(folder, image_file), 'r+b') as f:
            with Image.open(f) as image:
                image = resize_image(image)
                image.save(os.path.join(resized_folder, image_file), image.format)
        if i % 100 == 0:
            print('Resized images: %d/%d' %(i, num_images))

Start resizing train images.
Resized images: 0/82783
Resized images: 100/82783
Resized images: 200/82783
Resized images: 300/82783
Resized images: 400/82783
Resized images: 500/82783
Resized images: 600/82783
Resized images: 700/82783
Resized images: 800/82783
Resized images: 900/82783
Resized images: 1000/82783
Resized images: 1100/82783
Resized images: 1200/82783
Resized images: 1300/82783
Resized images: 1400/82783
Resized images: 1500/82783
Resized images: 1600/82783
Resized images: 1700/82783
Resized images: 1800/82783
Resized images: 1900/82783
Resized images: 2000/82783
Resized images: 2100/82783
Resized images: 2200/82783
Resized images: 2300/82783
Resized images: 2400/82783
Resized images: 2500/82783
Resized images: 2600/82783
Resized images: 2700/82783
Resized images: 2800/82783
Resized images: 2900/82783
Resized images: 3000/82783
Resized images: 3100/82783
Resized images: 3200/82783
Resized images: 3300/82783
Resized images: 3400/82783
Resized images: 3500/82783
Resized ima

In [ ]:
def process_caption_data_train(caption_file, image_dir, max_length):
    with open(caption_file) as f:
        caption_data = json.load(f)

    # id_to_filename is a dictionary such as {image_id: filename]} 
    id_to_filename = {image['id']: image['file_name'] for image in caption_data['images']}

    # data is a list of dictionary which contains 'captions', 'file_name' and 'image_id' as key.
    data = []
    for annotation in caption_data['annotations']:
        image_id = annotation['image_id']
        annotation['file_name'] = os.path.join(image_dir, id_to_filename[image_id])
        data += [annotation]
# convert to pandas dataframe (for later visualization or debugging)
    caption_data = pd.DataFrame.from_dict(data)
    del caption_data['id']
    caption_data.sort_values(by='image_id', inplace=True)
    caption_data = caption_data.reset_index(drop=True)
    
    del_idx = []
    for i, caption in enumerate(caption_data['caption']):
        caption = caption.replace('.','').replace(',','').replace("'","").replace('"','')
        caption = caption.replace('&','and').replace('(','').replace(")","").replace('-',' ')
        caption = " ".join(caption.split())  # replace multiple spaces
        
        caption_data.at[i, 'caption' ]=caption.lower()
        if len(caption.split(" ")) > max_length:
            del_idx.append(i)
    
    # delete captions if size is larger than max_length
    print("The number of captions before deletion: %d" %len(caption_data))
    caption_data = caption_data.drop(caption_data.index[del_idx])
    caption_data = caption_data.reset_index(drop=True)
    print("The number of captions after deletion: %d" %len(caption_data))
    return caption_data


In [ ]:
def process_caption_data_val(caption_file, image_dir, max_length):
    with open(caption_file) as f:
        caption_data = json.load(f)

    # id_to_filename is a dictionary such as {image_id: filename]} 
    id_to_filename = {image['id']: image['file_name'] for image in caption_data['images']}

    # data is a list of dictionary which contains 'captions', 'file_name' and 'image_id' as key.
    data = []
    for annotation in caption_data['annotations']:
        image_id = annotation['image_id']
        annotation['file_name'] = os.path.join(image_dir, id_to_filename[image_id])
        data += [annotation]
# convert to pandas dataframe (for later visualization or debugging)
    caption_data = pd.DataFrame.from_dict(data)
    del caption_data['id']
    caption_data.sort_values(by='image_id', inplace=True)
    caption_data = caption_data.reset_index(drop=True)
    
    del_idx = []
    for i, caption in enumerate(caption_data['caption']):
        caption = caption.replace('.','').replace(',','').replace("'","").replace('"','')
        caption = caption.replace('&','and').replace('(','').replace(")","").replace('-',' ')
        caption = " ".join(caption.split())  # replace multiple spaces
        
        caption_data.at[i, 'caption' ]=caption.lower()
        if len(caption.split(" ")) > max_length:
            del_idx.append(i)
    
    # delete captions if size is larger than max_length
    print("The number of captions before deletion: %d" %len(caption_data))
    caption_data = caption_data.drop(caption_data.index[del_idx])
    caption_data = caption_data.reset_index(drop=True)
    print("The number of captions after deletion: %d" %len(caption_data))
    return caption_data


In [ ]:
def build_vocab(annotations, threshold=1):
    counter = Counter()
    max_len = 0
    for i, caption in enumerate(annotations['caption']):
        words = caption.split(' ') # caption contrains only lower-case words
        for w in words:
            counter[w] +=1
        
        if len(caption.split(" ")) > max_len:
            max_len = len(caption.split(" "))

    vocab = [word for word in counter if counter[word] >= threshold]
    print('Filtered %d words to %d words with word count threshold %d.' % (len(counter), len(vocab), threshold))

    word_to_idx = {u'<NULL>': 0, u'<START>': 1, u'<END>': 2}
    idx = 3
    for word in vocab:
        word_to_idx[word] = idx
        idx += 1
    print("Max length of caption: ", max_len)
    return word_to_idx

In [ ]:
def build_caption_vector(annotations, word_to_idx, max_length=15):
    n_examples = len(annotations)
    captions = np.ndarray((n_examples,max_length+2)).astype(np.int32)   

    for i, caption in enumerate(annotations['caption']):
        words = caption.split(" ") # caption contrains only lower-case words
        cap_vec = []
        cap_vec.append(word_to_idx['<START>'])
        for word in words:
            if word in word_to_idx:
                cap_vec.append(word_to_idx[word])
        cap_vec.append(word_to_idx['<END>'])
        
        # pad short caption with the special null token '<NULL>' to make it fixed-size vector
        if len(cap_vec) < (max_length + 2):
            for j in range(max_length + 2 - len(cap_vec)):
                cap_vec.append(word_to_idx['<NULL>']) 
        
        captions[i, :] = np.asarray(cap_vec)
    print("Finished building caption vectors")
    return captions

In [ ]:
def build_file_names_train(annotations):
    image_file_names = []
    id_to_idx = {}
    idx = 0
    image_ids = annotations['image_id']
    file_names = annotations['file_name']
    for image_id, file_name in zip(image_ids, file_names):
        if not image_id in id_to_idx:
            id_to_idx[image_id] = idx
            image_file_names.append(file_name)
            idx += 1

    file_names = np.asarray(image_file_names)
    return file_names, id_to_idx

In [ ]:
def build_file_names_val(annotations):
    image_file_names = []
    id_to_idx = {}
    idx = 0
    image_ids = annotations['image_id']
    file_names = annotations['file_name']
    for image_id, file_name in zip(image_ids, file_names):
        if not image_id in id_to_idx:
            id_to_idx[image_id] = idx
            image_file_names.append(file_name)
            idx += 1

    file_names = np.asarray(image_file_names)
    return file_names, id_to_idx

In [ ]:
def build_image_idxs_train(annotations, id_to_idx):
    image_idxs = np.ndarray(len(annotations), dtype=np.int32)
    image_ids = annotations['image_id']
    for i, image_id in enumerate(image_ids):
        image_idxs[i] = id_to_idx[image_id]
    return image_idxs

In [ ]:
def build_image_idxs_val(annotations, id_to_idx):
    image_idxs = np.ndarray(len(annotations), dtype=np.int32)
    image_ids = annotations['image_id']
    for i, image_id in enumerate(image_ids):
        image_idxs[i] = id_to_idx[image_id]
    return image_idxs

In [ ]:
vgg_layers = ['conv1_1', 'relu1_1', 'conv1_2', 'relu1_2', 'pool1',
              'conv2_1', 'relu2_1', 'conv2_2', 'relu2_2', 'pool2',
              'conv3_1', 'relu3_1', 'conv3_2', 'relu3_2', 'conv3_3', 'relu3_3', 'conv3_4', 'relu3_4', 'pool3',
              'conv4_1', 'relu4_1', 'conv4_2', 'relu4_2', 'conv4_3', 'relu4_3', 'conv4_4', 'relu4_4', 'pool4',
              'conv5_1', 'relu5_1', 'conv5_2', 'relu5_2', 'conv5_3', 'relu5_3', 'conv5_4', 'relu5_4']


class Vgg19(object):
    def __init__(self, vgg_path):
        self.vgg_path = vgg_path

    def build_inputs(self):
        self.images = tf.keras.Input(dtype=tf.float32, shape=[224, 224, 3], name='images')

    def build_params(self):
        model = scipy.io.loadmat(self.vgg_path)
        layers = model['layers'][0]
        self.params = {}
        with tf.variable_scope('encoder'):
            for i, layer in enumerate(layers):
                layer_name = layer[0][0][0][0]
                layer_type = layer[0][0][1][0]
                if layer_type == 'conv':
                    w = layer[0][0][2][0][0].transpose(1, 0, 2, 3)
                    b = layer[0][0][2][0][1].reshape(-1)
                    self.params[layer_name] = {}
                    self.params[layer_name]['w'] = tf.get_variable(layer_name+'/w', initializer=tf.constant(w))
                    self.params[layer_name]['b'] = tf.get_variable(layer_name+'/b',initializer=tf.constant(b))

    def _conv(self, x, w, b):
        return tf.nn.bias_add(tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding='SAME'), b)

    def _relu(self, x):
        return tf.nn.relu(x)

    def _pool(self, x):
        return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    def build_model(self):
        for i, layer in enumerate(vgg_layers):
            layer_type = layer[:4]
            if layer_type == 'conv':
                if layer == 'conv1_1':
                    h = self.images
                h = self._conv(h, self.params[layer]['w'], self.params[layer]['b'])
            elif layer_type == 'relu':
                h = self._relu(h)
            elif layer_type == 'pool':
                h = self._pool(h)
            if layer == 'conv5_3':
                self.features = tf.reshape(h, [-1, 196, 512])
            

    def build(self):
        self.build_inputs()
        self.build_params()
        self.build_model()

In [ ]:
len(list(train_dataset['file_name'][200000:].unique()))

41412

In [ ]:
annotations

,image_id,caption,file_name
0,9,closeup of bins of food that include broccoli ...,image/train2014_resized/COCO_train2014_0000000...
1,9,a meal is presented in brightly colored plasti...,image/train2014_resized/COCO_train2014_0000000...
2,9,there are containers filled with different kin...,image/train2014_resized/COCO_train2014_0000000...
3,9,a bunch of trays that have different food,image/train2014_resized/COCO_train2014_0000000...
4,9,colorful dishes holding meat vegetables fruit ...,image/train2014_resized/COCO_train2014_0000000...
...,...,...,...
399993,581921,a snowboarder is in the air as he attempts a s...,image/train2014_resized/COCO_train2014_0000005...
399994,581921,a snowboarder jumping high on a snowy day on t...,image/train2014_resized/COCO_train2014_0000005...
399995,581921,there is a man that is jumping high on a snow ...,image/train2014_resized/COCO_train2014_0000005...
399996,581921,a man riding an orange snow board jumping off ...,image/train2014_resized/COCO_train2014_0000005...


In [ ]:
tf.reset_default_graph()

# if not os.path.exists('/content/data/test'): 
#   os.mkdir('/content/data/train')
#   os.mkdir('/content/data/val')
#   os.mkdir('/content/data/test')
# else:
#   pass
  
# # batch size for extracting feature vectors from vggnet.
# batch_size = 100
# # maximum length of caption(number of word). if caption is longer than max_length, deleted.  
# max_length = 15
# # if word occurs less than word_count_threshold in training dataset, the word index is special unknown token.
# word_count_threshold = 1
# # vgg model path 
# vgg_model_path = './data/imagenet-vgg-verydeep-19.mat'

# caption_file = 'data/annotations/captions_train2014.json'
# image_dir = 'image/%2014_resized/'

# # about 80000 images and 400000 captions for train dataset
# train_dataset = process_caption_data_train(caption_file='data/annotations/captions_train2014.json',
#                                       image_dir='image/train2014_resized/',
#                                       max_length=max_length)

# # about 40000 images and 200000 captions
# val_dataset = process_caption_data_val(caption_file='data/annotations/captions_val2014.json',
#                                     image_dir='image/val2014_resized/',
#                                     max_length=max_length)

# # about 4000 images and 20000 captions for val / test dataset
# val_cutoff = int(0.1 * len(val_dataset))
# test_cutoff = int(0.2 * len(val_dataset))
# print('Finished processing caption data')

# save_pickle(train_dataset, 'data/train/train.annotations.pkl')
# save_pickle(val_dataset[:val_cutoff], 'data/val/val.annotations.pkl')
# save_pickle(val_dataset[val_cutoff:test_cutoff].reset_index(drop=True), 'data/test/test.annotations.pkl')

# for split in ['train', 'val', 'test']:
#     annotations = load_pickle('./data/%s/%s.annotations.pkl' % (split, split))

#     if split == 'train':
#         word_to_idx = build_vocab(annotations=annotations, threshold=word_count_threshold)
#         save_pickle(word_to_idx, './data/%s/word_to_idx.pkl' % split)
    
#     captions = build_caption_vector(annotations=annotations, word_to_idx=word_to_idx, max_length=max_length)
#     save_pickle(captions, './data/%s/%s.captions.pkl' % (split, split))

#     file_names, id_to_idx = build_file_names_train(annotations)
#     save_pickle(file_names, './data/%s/%s.file.names.pkl' % (split, split))

#     image_idxs = build_image_idxs_train(annotations, id_to_idx)
#     save_pickle(image_idxs, './data/%s/%s.image.idxs.pkl' % (split, split))

#     # prepare reference captions to compute bleu scores later
#     image_ids = {}
#     feature_to_captions = {}
#     i = -1
#     for caption, image_id in zip(annotations['caption'], annotations['image_id']):
#         if not image_id in image_ids:
#             image_ids[image_id] = 0
#             i += 1
#             feature_to_captions[i] = []
#         feature_to_captions[i].append(caption.lower() + ' .')
#     save_pickle(feature_to_captions, './data/%s/%s.references.pkl' % (split, split))
#     print("Finished building %s caption dataset" %split)
    
# extract conv5_3 feature vectors
vggnet = Vgg19('/content/data/imagenet-vgg-verydeep-19.mat')
vggnet.build()
with tf.compat.v1.Session() as sess:
    tf.compat.v1.initialize_all_variables().run()
    for split in ['test']:
        anno_path = './data/%s/%s.annotations.pkl' % (split, split)
        save_path = './data/%s/%s.features.hkl' % (split, split)
        annotations = load_pickle(anno_path)
        image_path = list(annotations['file_name'].unique()[:1000])
        n_examples = len(image_path)

        all_feats = np.ndarray([n_examples, 196, 512], dtype=np.float32)

        for start, end in zip(range(0, n_examples, batch_size),
                              range(batch_size, n_examples + batch_size, batch_size)):
            image_batch_file = image_path[start:end]
            image_batch = np.array(list(map(lambda x: imageio.imread(x, as_gray=False, pilmode='RGB'), image_batch_file))).astype(
                np.float32)
            feats = sess.run(vggnet.features, feed_dict={vggnet.images: image_batch})
            all_feats[start:end, :] = feats
            print("Processed %d %s features.." % (end, split))

        # use hickle to save huge feature vectors
        hickle.dump(all_feats, save_path)
        print("Saved %s.." % (save_path))

Loaded ./data/test/test.annotations.pkl..
Processed 100 test features..
Processed 200 test features..
Processed 300 test features..
Processed 400 test features..
Processed 500 test features..
Processed 600 test features..
Processed 700 test features..
Processed 800 test features..
Processed 900 test features..
Processed 1000 test features..
Saved ./data/test/test.features.hkl..


In [ ]:
data = load_coco_data(data_path='./data', split='train')

word_to_idx = data['word_to_idx']

val_data = load_coco_data(data_path='./data', split='val')

features <class 'numpy.ndarray'> (20000, 196, 512) float32
file_names <class 'numpy.ndarray'> (82783,) <U55
captions <class 'numpy.ndarray'> (399998, 17) int32
image_idxs <class 'numpy.ndarray'> (399998,) int32
word_to_idx <class 'dict'> 23110
Elapse time: 111.70
features <class 'numpy.ndarray'> (4052, 196, 512) float32
file_names <class 'numpy.ndarray'> (4052,) <U51
captions <class 'numpy.ndarray'> (19589, 17) int32
image_idxs <class 'numpy.ndarray'> (19589,) int32
Elapse time: 22.49


In [ ]:
class CaptionGenerator(object):
    def __init__(self, word_to_idx, dim_feature=[196, 512], dim_embed=512, dim_hidden=1024, n_time_step=16,
                  prev2out=True, ctx2out=True, alpha_c=0.0, selector=True, dropout=True):
        """
        Args:
            word_to_idx: word-to-index mapping dictionary.
            dim_feature: (optional) Dimension of vggnet19 conv5_3 feature vectors.
            dim_embed: (optional) Dimension of word embedding.
            dim_hidden: (optional) Dimension of all hidden state.
            n_time_step: (optional) Time step size of LSTM.
            prev2out: (optional) previously generated word to hidden state. (see Eq (7) for explanation)
            ctx2out: (optional) context to hidden state (see Eq (7) for explanation)
            alpha_c: (optional) Doubly stochastic regularization coefficient. (see Section (4.2.1) for explanation)
            selector: (optional) gating scalar for context vector. (see Section (4.2.1) for explanation)
            dropout: (optional) If true then dropout layer is added.
        """

        self.word_to_idx = word_to_idx
        self.idx_to_word = {i: w for w, i in word_to_idx.items()}
        self.prev2out = prev2out
        self.ctx2out = ctx2out
        self.alpha_c = alpha_c
        self.selector = selector
        self.dropout = dropout
        self.V = len(word_to_idx)
        self.L = dim_feature[0]
        self.D = dim_feature[1]
        self.M = dim_embed
        self.H = dim_hidden
        self.T = n_time_step
        self._start = word_to_idx['<START>']
        self._null = word_to_idx['<NULL>']

        self.weight_initializer = tf.contrib.layers.xavier_initializer()
        self.const_initializer = tf.constant_initializer(0.0)
        self.emb_initializer = tf.random_uniform_initializer(minval=-1.0, maxval=1.0)

        # Place holder for features and captions
        self.features = tf.placeholder(tf.float32, [None, self.L, self.D])
        self.captions = tf.placeholder(tf.int32, [None, self.T + 1])

    def _get_initial_lstm(self, features):
        with tf.variable_scope('initial_lstm'):
            features_mean = tf.reduce_mean(features, 1)

            w_h = tf.get_variable('w_h', [self.D, self.H], initializer=self.weight_initializer)
            b_h = tf.get_variable('b_h', [self.H], initializer=self.const_initializer)
            h = tf.nn.tanh(tf.matmul(features_mean, w_h) + b_h)

            w_c = tf.get_variable('w_c', [self.D, self.H], initializer=self.weight_initializer)
            b_c = tf.get_variable('b_c', [self.H], initializer=self.const_initializer)
            c = tf.nn.tanh(tf.matmul(features_mean, w_c) + b_c)
            return c, h

    def _word_embedding(self, inputs, reuse=False):
        with tf.variable_scope('word_embedding', reuse=reuse):
            w = tf.get_variable('w', [self.V, self.M], initializer=self.emb_initializer)
            x = tf.nn.embedding_lookup(w, inputs, name='word_vector')  # (N, T, M) or (N, M)
            return x

    def _project_features(self, features):
        with tf.variable_scope('project_features'):
            w = tf.get_variable('w', [self.D, self.D], initializer=self.weight_initializer)
            features_flat = tf.reshape(features, [-1, self.D])
            features_proj = tf.matmul(features_flat, w)
            features_proj = tf.reshape(features_proj, [-1, self.L, self.D])
            return features_proj

    def _attention_layer(self, features, features_proj, h, reuse=False):
        with tf.variable_scope('attention_layer', reuse=reuse):
            w = tf.get_variable('w', [self.H, self.D], initializer=self.weight_initializer)
            b = tf.get_variable('b', [self.D], initializer=self.const_initializer)
            w_att = tf.get_variable('w_att', [self.D, 1], initializer=self.weight_initializer)

            h_att = tf.nn.relu(features_proj + tf.expand_dims(tf.matmul(h, w), 1) + b)    # (N, L, D)
            out_att = tf.reshape(tf.matmul(tf.reshape(h_att, [-1, self.D]), w_att), [-1, self.L])   # (N, L)
            alpha = tf.nn.softmax(out_att)
            context = tf.reduce_sum(features * tf.expand_dims(alpha, 2), 1, name='context')   #(N, D)
            return context, alpha

    def _selector(self, context, h, reuse=False):
        with tf.variable_scope('selector', reuse=reuse):
            w = tf.get_variable('w', [self.H, 1], initializer=self.weight_initializer)
            b = tf.get_variable('b', [1], initializer=self.const_initializer)
            beta = tf.nn.sigmoid(tf.matmul(h, w) + b, 'beta')    # (N, 1)
            context = tf.multiply(beta, context, name='selected_context')
            return context, beta

    def _decode_lstm(self, x, h, context, dropout=False, reuse=False):
        with tf.variable_scope('logits', reuse=reuse):
            w_h = tf.get_variable('w_h', [self.H, self.M], initializer=self.weight_initializer)
            b_h = tf.get_variable('b_h', [self.M], initializer=self.const_initializer)
            w_out = tf.get_variable('w_out', [self.M, self.V], initializer=self.weight_initializer)
            b_out = tf.get_variable('b_out', [self.V], initializer=self.const_initializer)

            if dropout:
                h = tf.nn.dropout(h, 0.5)
            h_logits = tf.matmul(h, w_h) + b_h

            if self.ctx2out:
                w_ctx2out = tf.get_variable('w_ctx2out', [self.D, self.M], initializer=self.weight_initializer)
                h_logits += tf.matmul(context, w_ctx2out)

            if self.prev2out:
                h_logits += x
            h_logits = tf.nn.tanh(h_logits)

            if dropout:
                h_logits = tf.nn.dropout(h_logits, 0.5)
            out_logits = tf.matmul(h_logits, w_out) + b_out
            return out_logits

    def _batch_norm(self, x, mode='train', name=None):
        return tf.contrib.layers.batch_norm(inputs=x,
                                            decay=0.95,
                                            center=True,
                                            scale=True,
                                            is_training=(mode=='train'),
                                            updates_collections=None,
                                            scope=(name+'batch_norm'))

    def build_model(self):
        features = self.features
        captions = self.captions
        batch_size = tf.shape(features)[0]

        captions_in = captions[:, :self.T]
        captions_out = captions[:, 1:]
        mask = tf.to_float(tf.not_equal(captions_out, self._null))


        # batch normalize feature vectors
        features = self._batch_norm(features, mode='train', name='conv_features')

        c, h = self._get_initial_lstm(features=features)
        x = self._word_embedding(inputs=captions_in)
        features_proj = self._project_features(features=features)

        loss = 0.0
        alpha_list = []
        lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=self.H)

        for t in range(self.T):
            context, alpha = self._attention_layer(features, features_proj, h, reuse=(t!=0))
            alpha_list.append(alpha)

            if self.selector:
                context, beta = self._selector(context, h, reuse=(t!=0))

            with tf.variable_scope('lstm', reuse=(t!=0)):
                _, (c, h) = lstm_cell(inputs=tf.concat( [x[:,t,:], context],1), state=[c, h])

            logits = self._decode_lstm(x[:,t,:], h, context, dropout=self.dropout, reuse=(t!=0))

            loss += tf.reduce_sum(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=captions_out[:, t],logits=logits)*mask[:, t] )

        if self.alpha_c > 0:
            alphas = tf.transpose(tf.stack(alpha_list), (1, 0, 2))     # (N, T, L)
            alphas_all = tf.reduce_sum(alphas, 1)      # (N, L)
            alpha_reg = self.alpha_c * tf.reduce_sum((16./196 - alphas_all) ** 2)
            loss += alpha_reg

        return loss / tf.to_float(batch_size)

    def build_sampler(self, max_len=20):
        features = self.features

        # batch normalize feature vectors
        features = self._batch_norm(features, mode='test', name='conv_features')

        c, h = self._get_initial_lstm(features=features)
        features_proj = self._project_features(features=features)

        sampled_word_list = []
        alpha_list = []
        beta_list = []
        lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=self.H)

        for t in range(max_len):
            if t == 0:
                x = self._word_embedding(inputs=tf.fill([tf.shape(features)[0]], self._start))
            else:
                x = self._word_embedding(inputs=sampled_word, reuse=True)

            context, alpha = self._attention_layer(features, features_proj, h, reuse=(t!=0))
            alpha_list.append(alpha)

            if self.selector:
                context, beta = self._selector(context, h, reuse=(t!=0))
                beta_list.append(beta)

            with tf.variable_scope('lstm', reuse=(t!=0)):
                _, (c, h) = lstm_cell(inputs=tf.concat( [x, context],1), state=[c, h])

            logits = self._decode_lstm(x, h, context, reuse=(t!=0))
            sampled_word = tf.argmax(logits, 1)
            sampled_word_list.append(sampled_word)

        alphas = tf.transpose(tf.stack(alpha_list), (1, 0, 2))     # (N, T, L)
        betas = tf.transpose(tf.squeeze(beta_list), (1, 0))    # (N, T)
        sampled_captions = tf.transpose(tf.stack(sampled_word_list), (1, 0))     # (N, max_len)
        return alphas, betas, sampled_captions


In [ ]:
model = CaptionGenerator(word_to_idx, dim_feature=[196, 512], dim_embed=512,
                                       dim_hidden=1024, n_time_step=16, prev2out=True, 
                                                 ctx2out=True, alpha_c=1.0, selector=True, dropout=True)

In [ ]:
solver = CaptioningSolver(model, data, val_data, n_epochs=20, batch_size=128, update_rule='adam',
                                          learning_rate=0.001, print_every=1000, save_every=1, image_path='./image/',
                                    pretrained_model='model/lstm/model-16.index', model_path='model/lstm/',
                                     print_bleu=True, log_path='log/')

In [ ]:
solver.train()



Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



The number of epoch: 20
Data size: 20000
Batch size: 128
Iterations per epoch: 157




Previous epoch loss:  -1
Current epoch loss:  9314.324810028076
Elapsed time:  1024.2937955856323
Saved ./data/val/val.candidate.captions.pkl..
{'testlen': 37195, 'reflen': 39220, 'guess': [37195, 33143, 29091, 25039], 'correct': [19446, 4393, 707, 196]}
ratio: 0.9483681795002309
Bl

In [ ]:
plt.rcParams['figure.figsize'] = (8.0, 6.0)  # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
%load_ext autoreload
%autoreload 2

In [ ]:
solver.test(data, split='val')

In [ ]:
data = load_coco_data(data_path='/content/data', split='val')
with open('/content/data/train/word_to_idx.pkl', 'rb') as f:
    word_to_idx = pickle.load(f)

features <class 'numpy.ndarray'> (4052, 196, 512) float32
file_names <class 'numpy.ndarray'> (4052,) <U51
captions <class 'numpy.ndarray'> (19589, 17) int32
image_idxs <class 'numpy.ndarray'> (19589,) int32
Elapse time: 22.45


In [ ]:
model = CaptionGenerator(word_to_idx, dim_feature=[196, 512], dim_embed=512,
                                   dim_hidden=1500, n_time_step=16, prev2out=True, 
                                             ctx2out=True, alpha_c=1.0, selector=True, dropout=True)

In [ ]:
solver = CaptioningSolver(model, data, data, n_epochs=15, batch_size=128, update_rule='adam',
                                      learning_rate=0.0025, print_every=2000, save_every=1, image_path='./image/val2014_resized',
                                pretrained_model='/content/model/lstm/model-16.index', model_path='./model/lstm',
                                 print_bleu=False, log_path='./log/')

In [ ]:
# tf.get_variable_scope().reuse_variables()
solver.test(data, split='val')